# Main Project


## Data Extraction

In [1]:
import requests

url = 'https://sigel.aneel.gov.br/arcgis/rest/services/PORTAL/WFS/MapServer/0/query'

response = requests.get(url)

print('url: ', response.url)

url:  https://sigel.aneel.gov.br/arcgis/rest/services/PORTAL/WFS/MapServer/0/query


In [2]:
print('Status code: ', response.status_code)

Status code:  200


In [3]:
params = {
    "where": "1=1",    
    "outFields": "*", 
    "f": "json",
    #"resultRecordCount": 5 
} 

response = requests.get(url, params=params)
if response.status_code == 200:
    try:
        data = response.json()  # Converter para JSON
        #print(data)
    except ValueError:  # JSON inválido
        print("Resposta não é um JSON válido.")
else:
    print(f"Erro na requisição: {response.status_code}")



## Data Processing

### Inicializando o .CSV

In [4]:
import geopandas as gpd

In [5]:
json_data = data

In [6]:
json_data['features'][0]['geometry']

{'x': -42.642494529999965, 'y': -14.470967592999955}

In [7]:
long = [feature['geometry']['x'] for feature in json_data['features']]
lat = [feature['geometry']['y'] for feature in json_data['features']]

In [8]:
gdf_coordinates = gpd.GeoDataFrame(
    {'Longitude': long, 'Latitude': lat},
    geometry=gpd.points_from_xy(long,lat),
    crs="EPSG:4326"
)

gdf_coordinates.head(5)

,Longitude,Latitude,geometry
0,-42.642495,-14.470968,POINT (-42.64249 -14.47097)
1,-35.924469,-5.115754,POINT (-35.92447 -5.11575)
2,-35.844373,-5.267424,POINT (-35.84437 -5.26742)
3,-35.923294,-5.442082,POINT (-35.92329 -5.44208)
4,-38.879826,-3.540330,POINT (-38.87983 -3.54033)


In [9]:
features = json_data['features']
attributes = [feature['attributes'] for feature in features]
gdf = gpd.GeoDataFrame(
    attributes,
    geometry=gpd.points_from_xy(
        long,
        lat
    ),
    crs="EPSG:4326"
)

gdf.columns

Index(['POT_MW', 'ALT_TOTAL', 'ALT_TORRE', 'DIAM_ROTOR', 'DATA_ATUALIZACAO',
       'EOL_VERSAO_ID', 'NOME_EOL', 'DEN_AEG', 'X', 'Y', 'VERSAO', 'DATUM_EMP',
       'OPERACAO', 'FUSO_AG', 'PROPRIETARIO', 'ORIGEM', 'OBJECTID', 'UF',
       'CEG', 'geometry'],
      dtype='object')

In [10]:
gdf.head(3)


,POT_MW,ALT_TOTAL,ALT_TORRE,DIAM_ROTOR,DATA_ATUALIZACAO,EOL_VERSAO_ID,NOME_EOL,DEN_AEG,X,Y,VERSAO,DATUM_EMP,OPERACAO,FUSO_AG,PROPRIETARIO,ORIGEM,OBJECTID,UF,CEG,geometry
0,1.85,121.25,80.0,82.5,1666625847000,30739,Caititu,CAI-083,754125.0,8398879.0,Versão Válida,WGS84,Sim,UTM 23 S (MER -45),Central Eólica Caititu S.A.,None,1,BA,EOLCVBA030739-4-01,POINT (-42.64249 -14.47097)
1,2.00,145.00,95.0,100.0,1666625847000,30512,Dreen Boa Vista,BV07,175728.0,9433802.0,Versão Válida,SIRGAS2000,Sim,UTM 25 S (MER -33),GE Boa Vista S.A.,None,2,RN,EOLCVRN030512-0-01,POINT (-35.92447 -5.11575)
2,2.00,145.00,95.0,100.0,1666625847000,30497,Renascença V,15,184692.0,9417056.0,Versão Válida,SIRGAS2000,Sim,UTM 25 S (MER -33),Renascença V Energias Renováveis S.A.,None,3,RN,EOLCVRN030497-2-01,POINT (-35.84437 -5.26742)


In [11]:
# Adicionando novas colunas ao GeoDataFrame
gdf['LONG'] = long
gdf['LAT'] = lat

In [12]:
gdf.head(-5)

,POT_MW,ALT_TOTAL,ALT_TORRE,DIAM_ROTOR,DATA_ATUALIZACAO,EOL_VERSAO_ID,NOME_EOL,DEN_AEG,X,Y,...,OPERACAO,FUSO_AG,PROPRIETARIO,ORIGEM,OBJECTID,UF,CEG,geometry,LONG,LAT
0,1.85,121.25,80.00,82.5,1666625847000,30739,Caititu,CAI-083,754125.00,8398879.00,...,Sim,UTM 23 S (MER -45),Central Eólica Caititu S.A.,None,1,BA,EOLCVBA030739-4-01,POINT (-42.64249 -14.47097),-42.642495,-14.470968
1,2.00,145.00,95.00,100.0,1666625847000,30512,Dreen Boa Vista,BV07,175728.00,9433802.00,...,Sim,UTM 25 S (MER -33),GE Boa Vista S.A.,None,2,RN,EOLCVRN030512-0-01,POINT (-35.92447 -5.11575),-35.924469,-5.115754
2,2.00,145.00,95.00,100.0,1666625847000,30497,Renascença V,15,184692.00,9417056.00,...,Sim,UTM 25 S (MER -33),Renascença V Energias Renováveis S.A.,None,3,RN,EOLCVRN030497-2-01,POINT (-35.84437 -5.26742),-35.844373,-5.267424
3,2.70,150.00,89.00,122.0,1666625847000,30864,SM,1,176028.00,9397684.00,...,Sim,UTM 25 S (MER -33),Santa Maria Energias Renováveis S.A.,None,4,RN,EOLCVRN030864-1-01,POINT (-35.92329 -5.44208),-35.923294,-5.442082
4,2.10,143.70,96.20,95.0,1666625847000,30295,Colônia,AEG-08,513347.87,9608681.26,...,Sim,UTM 24 S (MER -39),Central Geradora Eólica Colônia S.A.,None,5,CE,EOLCVCE030295-3-01,POINT (-38.87983 -3.54033),-38.879826,-3.540330
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
990,2.00,138.50,90.00,97.0,1666625847000,31338,Damascena,Damascena 07,229993.00,8686763.00,...,Sim,UTM 24 S (MER -39),Enel Green Power Damascena Eólica S.A.,None,991,BA,EOLCVBA031338-6-01,POINT (-41.47843 -11.86871),-41.478429,-11.868705
991,2.10,124.00,80.00,88.0,1666625847000,28699,Icaraizinho,AEG 04,425803.00,9665900.00,...,Sim,UTM 24 S (MER -39),CPFL Energias Renováveis S.A.,None,992,CE,EOLCVCE028699-0-01,POINT (-39.66769 -3.02246),-39.667695,-3.022463
992,2.00,149.38,108.38,82.0,1666625847000,30290,Mangue Seco 5,MS5-06,787884.00,9429755.00,...,Sim,UTM 24 S (MER -39),Eólica Mangue Seco 4 – Geradora e Comercializa...,None,993,RN,EOLCVRN030290-2-01,POINT (-36.40332 -5.15372),-36.403318,-5.153724
993,2.35,130.00,80.00,100.0,1666625847000,30806,Pedra do Gerônimo,AEG5,593977.00,8998246.00,...,Sim,UTM 24 S (MER -39),Enel Green Power Pedra do Gerônimo Eólica S.A.,None,994,PE,EOLCVPE030806-4-01,POINT (-38.14488 -9.06143),-38.144881,-9.061428


In [13]:
colunas_vazias = gdf.columns[gdf.isnull().all()]
gdf = gdf.drop(columns=colunas_vazias)

In [14]:
gdf.head()

,POT_MW,ALT_TOTAL,ALT_TORRE,DIAM_ROTOR,DATA_ATUALIZACAO,EOL_VERSAO_ID,NOME_EOL,DEN_AEG,X,Y,...,DATUM_EMP,OPERACAO,FUSO_AG,PROPRIETARIO,OBJECTID,UF,CEG,geometry,LONG,LAT
0,1.85,121.25,80.0,82.5,1666625847000,30739,Caititu,CAI-083,754125.00,8398879.00,...,WGS84,Sim,UTM 23 S (MER -45),Central Eólica Caititu S.A.,1,BA,EOLCVBA030739-4-01,POINT (-42.64249 -14.47097),-42.642495,-14.470968
1,2.00,145.00,95.0,100.0,1666625847000,30512,Dreen Boa Vista,BV07,175728.00,9433802.00,...,SIRGAS2000,Sim,UTM 25 S (MER -33),GE Boa Vista S.A.,2,RN,EOLCVRN030512-0-01,POINT (-35.92447 -5.11575),-35.924469,-5.115754
2,2.00,145.00,95.0,100.0,1666625847000,30497,Renascença V,15,184692.00,9417056.00,...,SIRGAS2000,Sim,UTM 25 S (MER -33),Renascença V Energias Renováveis S.A.,3,RN,EOLCVRN030497-2-01,POINT (-35.84437 -5.26742),-35.844373,-5.267424
3,2.70,150.00,89.0,122.0,1666625847000,30864,SM,1,176028.00,9397684.00,...,SIRGAS2000,Sim,UTM 25 S (MER -33),Santa Maria Energias Renováveis S.A.,4,RN,EOLCVRN030864-1-01,POINT (-35.92329 -5.44208),-35.923294,-5.442082
4,2.10,143.70,96.2,95.0,1666625847000,30295,Colônia,AEG-08,513347.87,9608681.26,...,SIRGAS2000,Sim,UTM 24 S (MER -39),Central Geradora Eólica Colônia S.A.,5,CE,EOLCVCE030295-3-01,POINT (-38.87983 -3.54033),-38.879826,-3.540330


In [15]:
gdf.columns[gdf.isnull().all()]

Index([], dtype='object')

In [16]:
#gdf = gdf.sort_values(by=["NOME_EOL", "DATA_ATUALIZACAO"])

In [17]:
gdf

,POT_MW,ALT_TOTAL,ALT_TORRE,DIAM_ROTOR,DATA_ATUALIZACAO,EOL_VERSAO_ID,NOME_EOL,DEN_AEG,X,Y,...,DATUM_EMP,OPERACAO,FUSO_AG,PROPRIETARIO,OBJECTID,UF,CEG,geometry,LONG,LAT
0,1.85,121.25,80.00,82.5,1666625847000,30739,Caititu,CAI-083,754125.0000,8.398879e+06,...,WGS84,Sim,UTM 23 S (MER -45),Central Eólica Caititu S.A.,1,BA,EOLCVBA030739-4-01,POINT (-42.64249 -14.47097),-42.642495,-14.470968
1,2.00,145.00,95.00,100.0,1666625847000,30512,Dreen Boa Vista,BV07,175728.0000,9.433802e+06,...,SIRGAS2000,Sim,UTM 25 S (MER -33),GE Boa Vista S.A.,2,RN,EOLCVRN030512-0-01,POINT (-35.92447 -5.11575),-35.924469,-5.115754
2,2.00,145.00,95.00,100.0,1666625847000,30497,Renascença V,15,184692.0000,9.417056e+06,...,SIRGAS2000,Sim,UTM 25 S (MER -33),Renascença V Energias Renováveis S.A.,3,RN,EOLCVRN030497-2-01,POINT (-35.84437 -5.26742),-35.844373,-5.267424
3,2.70,150.00,89.00,122.0,1666625847000,30864,SM,1,176028.0000,9.397684e+06,...,SIRGAS2000,Sim,UTM 25 S (MER -33),Santa Maria Energias Renováveis S.A.,4,RN,EOLCVRN030864-1-01,POINT (-35.92329 -5.44208),-35.923294,-5.442082
4,2.10,143.70,96.20,95.0,1666625847000,30295,Colônia,AEG-08,513347.8700,9.608681e+06,...,SIRGAS2000,Sim,UTM 24 S (MER -39),Central Geradora Eólica Colônia S.A.,5,CE,EOLCVCE030295-3-01,POINT (-38.87983 -3.54033),-38.879826,-3.540330
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,2.00,149.38,108.38,82.0,1666625847000,30285,Mangue Seco 1,MS1-09,798708.0000,9.425974e+06,...,SIRGAS2000,Sim,UTM 24 S (MER -39),Eólica Mangue Seco 1 – Geradora e Comercializa...,996,RN,EOLCVRN030285-6-01,POINT (-36.30599 -5.18791),-36.305990,-5.187911
996,2.00,149.38,108.00,82.0,1666625847000,30349,Cerro Chato II,AEG 9,625380.3248,6.586389e+06,...,SIRGAS2000,Sim,UTM 21 S (MER -57),Eletrosul Centrais Elétricas S.A.,997,RS,EOLCVRS030349-6-01,POINT (-55.68869 -30.84925),-55.688688,-30.849248
997,2.00,145.00,95.00,100.0,1666625847000,30527,Renascença III,UG9,177982.0380,9.413603e+06,...,SIRGAS2000,Sim,UTM 25 S (MER -33),Energisa Geração - Central Eólica Renascença I...,998,RN,EOLCVRN030527-8-01,POINT (-35.90497 -5.29835),-35.904970,-5.298351
998,1.60,121.25,80.00,82.5,1666625847000,30328,Guanambi,GUA_12,755777.0000,8.428659e+06,...,SAD69,Sim,UTM 23 S (MER -45),Centrais Eólicas Guanambi Ltda.,999,BA,EOLCVBA030328-3-01,POINT (-42.63041 -14.20218),-42.630413,-14.202182


In [18]:
gdf.drop(columns='geometry').to_csv("outputs/output.csv",index = False)

In [19]:
# Adicionando uma planilha separando as UF em setores do Brasil (Norte, Nordeste, Sul, Sudeste e Centro-Oeste)

setores = {
    "Norte": ["AC", "AM", "AP", "PA", "RO", "RR", "TO"],
    "Nordeste": ["AL", "BA", "CE", "MA", "PB", "PE", "PI", "RN", "SE"],
    "Centro-Oeste": ["DF", "GO", "MT", "MS"],
    "Sudeste": ["ES", "MG", "RJ", "SP"],
    "Sul": ["PR", "RS", "SC"]
}

def atribuir_setor(uf):
    for setor, ufs in setores.items():
        if uf in ufs:
            return setor
    return None

gdf['SETOR'] = gdf['UF'].apply(atribuir_setor)

In [20]:
gdf.head()

,POT_MW,ALT_TOTAL,ALT_TORRE,DIAM_ROTOR,DATA_ATUALIZACAO,EOL_VERSAO_ID,NOME_EOL,DEN_AEG,X,Y,...,OPERACAO,FUSO_AG,PROPRIETARIO,OBJECTID,UF,CEG,geometry,LONG,LAT,SETOR
0,1.85,121.25,80.0,82.5,1666625847000,30739,Caititu,CAI-083,754125.00,8398879.00,...,Sim,UTM 23 S (MER -45),Central Eólica Caititu S.A.,1,BA,EOLCVBA030739-4-01,POINT (-42.64249 -14.47097),-42.642495,-14.470968,Nordeste
1,2.00,145.00,95.0,100.0,1666625847000,30512,Dreen Boa Vista,BV07,175728.00,9433802.00,...,Sim,UTM 25 S (MER -33),GE Boa Vista S.A.,2,RN,EOLCVRN030512-0-01,POINT (-35.92447 -5.11575),-35.924469,-5.115754,Nordeste
2,2.00,145.00,95.0,100.0,1666625847000,30497,Renascença V,15,184692.00,9417056.00,...,Sim,UTM 25 S (MER -33),Renascença V Energias Renováveis S.A.,3,RN,EOLCVRN030497-2-01,POINT (-35.84437 -5.26742),-35.844373,-5.267424,Nordeste
3,2.70,150.00,89.0,122.0,1666625847000,30864,SM,1,176028.00,9397684.00,...,Sim,UTM 25 S (MER -33),Santa Maria Energias Renováveis S.A.,4,RN,EOLCVRN030864-1-01,POINT (-35.92329 -5.44208),-35.923294,-5.442082,Nordeste
4,2.10,143.70,96.2,95.0,1666625847000,30295,Colônia,AEG-08,513347.87,9608681.26,...,Sim,UTM 24 S (MER -39),Central Geradora Eólica Colônia S.A.,5,CE,EOLCVCE030295-3-01,POINT (-38.87983 -3.54033),-38.879826,-3.540330,Nordeste


In [21]:
gdf.columns[gdf.isnull().all()]

Index([], dtype='object')

In [22]:
gdf = gdf.drop(columns=gdf.columns[gdf.isnull().all()])

In [23]:
gdf.drop(columns='geometry').to_csv("outputs/output_setores.csv",index = False)

In [24]:
gdf['DATA_ATUALIZACAO']

0      1666625847000
1      1666625847000
2      1666625847000
3      1666625847000
4      1666625847000
           ...      
995    1666625847000
996    1666625847000
997    1666625847000
998    1666625847000
999    1666625847000
Name: DATA_ATUALIZACAO, Length: 1000, dtype: int64

In [25]:
gdf.drop(columns='geometry').to_csv("outputs/output.csv",index = False)

### DATA_ATUALIZACAO -> timestamp()

In [28]:
import pandas as pd
gdf['DATA_ATUALIZACAO'] = pd.to_datetime(gdf['DATA_ATUALIZACAO'], unit='ms')


In [29]:
gdf

,POT_MW,ALT_TOTAL,ALT_TORRE,DIAM_ROTOR,DATA_ATUALIZACAO,EOL_VERSAO_ID,NOME_EOL,DEN_AEG,X,Y,...,OPERACAO,FUSO_AG,PROPRIETARIO,OBJECTID,UF,CEG,geometry,LONG,LAT,SETOR
0,1.85,121.25,80.00,82.5,2022-10-24 15:37:27,30739,Caititu,CAI-083,754125.0000,8.398879e+06,...,Sim,UTM 23 S (MER -45),Central Eólica Caititu S.A.,1,BA,EOLCVBA030739-4-01,POINT (-42.64249 -14.47097),-42.642495,-14.470968,Nordeste
1,2.00,145.00,95.00,100.0,2022-10-24 15:37:27,30512,Dreen Boa Vista,BV07,175728.0000,9.433802e+06,...,Sim,UTM 25 S (MER -33),GE Boa Vista S.A.,2,RN,EOLCVRN030512-0-01,POINT (-35.92447 -5.11575),-35.924469,-5.115754,Nordeste
2,2.00,145.00,95.00,100.0,2022-10-24 15:37:27,30497,Renascença V,15,184692.0000,9.417056e+06,...,Sim,UTM 25 S (MER -33),Renascença V Energias Renováveis S.A.,3,RN,EOLCVRN030497-2-01,POINT (-35.84437 -5.26742),-35.844373,-5.267424,Nordeste
3,2.70,150.00,89.00,122.0,2022-10-24 15:37:27,30864,SM,1,176028.0000,9.397684e+06,...,Sim,UTM 25 S (MER -33),Santa Maria Energias Renováveis S.A.,4,RN,EOLCVRN030864-1-01,POINT (-35.92329 -5.44208),-35.923294,-5.442082,Nordeste
4,2.10,143.70,96.20,95.0,2022-10-24 15:37:27,30295,Colônia,AEG-08,513347.8700,9.608681e+06,...,Sim,UTM 24 S (MER -39),Central Geradora Eólica Colônia S.A.,5,CE,EOLCVCE030295-3-01,POINT (-38.87983 -3.54033),-38.879826,-3.540330,Nordeste
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,2.00,149.38,108.38,82.0,2022-10-24 15:37:27,30285,Mangue Seco 1,MS1-09,798708.0000,9.425974e+06,...,Sim,UTM 24 S (MER -39),Eólica Mangue Seco 1 – Geradora e Comercializa...,996,RN,EOLCVRN030285-6-01,POINT (-36.30599 -5.18791),-36.305990,-5.187911,Nordeste
996,2.00,149.38,108.00,82.0,2022-10-24 15:37:27,30349,Cerro Chato II,AEG 9,625380.3248,6.586389e+06,...,Sim,UTM 21 S (MER -57),Eletrosul Centrais Elétricas S.A.,997,RS,EOLCVRS030349-6-01,POINT (-55.68869 -30.84925),-55.688688,-30.849248,Sul
997,2.00,145.00,95.00,100.0,2022-10-24 15:37:27,30527,Renascença III,UG9,177982.0380,9.413603e+06,...,Sim,UTM 25 S (MER -33),Energisa Geração - Central Eólica Renascença I...,998,RN,EOLCVRN030527-8-01,POINT (-35.90497 -5.29835),-35.904970,-5.298351,Nordeste
998,1.60,121.25,80.00,82.5,2022-10-24 15:37:27,30328,Guanambi,GUA_12,755777.0000,8.428659e+06,...,Sim,UTM 23 S (MER -45),Centrais Eólicas Guanambi Ltda.,999,BA,EOLCVBA030328-3-01,POINT (-42.63041 -14.20218),-42.630413,-14.202182,Nordeste


In [31]:
gdf.drop(columns='geometry').to_csv("outputs/output.csv",index = False)

In [32]:
gdf

,POT_MW,ALT_TOTAL,ALT_TORRE,DIAM_ROTOR,DATA_ATUALIZACAO,EOL_VERSAO_ID,NOME_EOL,DEN_AEG,X,Y,...,OPERACAO,FUSO_AG,PROPRIETARIO,OBJECTID,UF,CEG,geometry,LONG,LAT,SETOR
0,1.85,121.25,80.00,82.5,2022-10-24 15:37:27,30739,Caititu,CAI-083,754125.0000,8.398879e+06,...,Sim,UTM 23 S (MER -45),Central Eólica Caititu S.A.,1,BA,EOLCVBA030739-4-01,POINT (-42.64249 -14.47097),-42.642495,-14.470968,Nordeste
1,2.00,145.00,95.00,100.0,2022-10-24 15:37:27,30512,Dreen Boa Vista,BV07,175728.0000,9.433802e+06,...,Sim,UTM 25 S (MER -33),GE Boa Vista S.A.,2,RN,EOLCVRN030512-0-01,POINT (-35.92447 -5.11575),-35.924469,-5.115754,Nordeste
2,2.00,145.00,95.00,100.0,2022-10-24 15:37:27,30497,Renascença V,15,184692.0000,9.417056e+06,...,Sim,UTM 25 S (MER -33),Renascença V Energias Renováveis S.A.,3,RN,EOLCVRN030497-2-01,POINT (-35.84437 -5.26742),-35.844373,-5.267424,Nordeste
3,2.70,150.00,89.00,122.0,2022-10-24 15:37:27,30864,SM,1,176028.0000,9.397684e+06,...,Sim,UTM 25 S (MER -33),Santa Maria Energias Renováveis S.A.,4,RN,EOLCVRN030864-1-01,POINT (-35.92329 -5.44208),-35.923294,-5.442082,Nordeste
4,2.10,143.70,96.20,95.0,2022-10-24 15:37:27,30295,Colônia,AEG-08,513347.8700,9.608681e+06,...,Sim,UTM 24 S (MER -39),Central Geradora Eólica Colônia S.A.,5,CE,EOLCVCE030295-3-01,POINT (-38.87983 -3.54033),-38.879826,-3.540330,Nordeste
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,2.00,149.38,108.38,82.0,2022-10-24 15:37:27,30285,Mangue Seco 1,MS1-09,798708.0000,9.425974e+06,...,Sim,UTM 24 S (MER -39),Eólica Mangue Seco 1 – Geradora e Comercializa...,996,RN,EOLCVRN030285-6-01,POINT (-36.30599 -5.18791),-36.305990,-5.187911,Nordeste
996,2.00,149.38,108.00,82.0,2022-10-24 15:37:27,30349,Cerro Chato II,AEG 9,625380.3248,6.586389e+06,...,Sim,UTM 21 S (MER -57),Eletrosul Centrais Elétricas S.A.,997,RS,EOLCVRS030349-6-01,POINT (-55.68869 -30.84925),-55.688688,-30.849248,Sul
997,2.00,145.00,95.00,100.0,2022-10-24 15:37:27,30527,Renascença III,UG9,177982.0380,9.413603e+06,...,Sim,UTM 25 S (MER -33),Energisa Geração - Central Eólica Renascença I...,998,RN,EOLCVRN030527-8-01,POINT (-35.90497 -5.29835),-35.904970,-5.298351,Nordeste
998,1.60,121.25,80.00,82.5,2022-10-24 15:37:27,30328,Guanambi,GUA_12,755777.0000,8.428659e+06,...,Sim,UTM 23 S (MER -45),Centrais Eólicas Guanambi Ltda.,999,BA,EOLCVBA030328-3-01,POINT (-42.63041 -14.20218),-42.630413,-14.202182,Nordeste
